# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": "sum"}).reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [5]:
df.columns

Index(['CustomerID', 'ProductName', 'Quantity'], dtype='object')

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
ptable = df.pivot_table(values='Quantity', columns=["CustomerID"], index=["ProductName"], aggfunc="sum").fillna(0)

In [7]:
ptable.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
matrix = squareform(pdist(ptable.T, 'euclidean'))
matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [9]:
distances = pd.DataFrame(1/(1 + squareform(pdist(ptable.T, 'euclidean'))), 
                         index=ptable.columns, columns=ptable.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [23]:
id_costumer = 33
similar33 = list(distances[id_costumer].sort_values(ascending=False)[1:6].index)
similar33

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [24]:
data_customers = data[data['CustomerID'].isin(similar33)]
data_customers.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
887,3305,Bernard,O'Neill,2025094,102,"Rosemary - Primerba, Paste",1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [16]:
grouped_customers = data_customers.groupby(["ProductName"]).agg({"Quantity": "sum"})
grouped_customers = grouped_customers.sort_values("Quantity", ascending=False)
grouped_customers.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [17]:
merged_data = grouped_customers.merge(ptable, on="ProductName") #ptable
merged_data.head(5)

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
Towels - Paper / Kraft,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


In [18]:
products = (merged_data.index[(merged_data["Quantity"] > 0) & (merged_data[33] == 0)].tolist())[:5]
products

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [19]:
recommendations = {}

In [20]:
customers_list = set([element for element in data["CustomerID"]])

In [25]:
for customer in customers_list:
    similars = list(distances[customer].sort_values(ascending=False)[1:6].head().index)
    sim_cust_prod = data[data['CustomerID'].isin(similars)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False)
    
    merged_data = ranked_products.merge(ptable, on="ProductName")
    products = (merged_data.index[(merged_data["Quantity"] > 0) & (merged_data[customer] == 0)].tolist())[:5]
    
    recommendations[customer] = products
                        

In [27]:
#recommendations

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [29]:
user_recommendations = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recommendations.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recommendations.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
2,92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
3,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
4,18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [33]:
matrix_cos = squareform(pdist(ptable.T, 'cosine'))
matrix_cos

array([[0.        , 0.88678126, 0.77464789, ..., 0.9223069 , 0.88864251,
        0.89702036],
       [0.88678126, 0.        , 0.86162154, ..., 0.8612137 , 0.90053882,
        0.89652438],
       [0.77464789, 0.86162154, 0.        , ..., 0.85756266, 0.80203113,
        0.89702036],
       ...,
       [0.9223069 , 0.8612137 , 0.85756266, ..., 0.        , 0.84074449,
        0.88165473],
       [0.88864251, 0.90053882, 0.80203113, ..., 0.84074449, 0.        ,
        0.83037577],
       [0.89702036, 0.89652438, 0.89702036, ..., 0.88165473, 0.83037577,
        0.        ]])

In [34]:
distances_cos = pd.DataFrame(1/(1 + squareform(pdist(ptable.T, 'cosine'))), 
                         index=ptable.columns, columns=ptable.columns)
distances_cos.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [38]:
recommendations_consine = {}

for customer in customers_list:
    similars_cos = list(distances_cos[customer].sort_values(ascending=False)[1:6].head().index)
    sim_cust_cos = data[data['CustomerID'].isin(similars_cos)]
    grouped_cos = sim_cust_cos.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products_cos = grouped_cos.sort_values('Quantity', ascending=False)
    
    merged_data_cos = ranked_products_cos.merge(ptable, on="ProductName")
    products_cos = (merged_data.index[(merged_data["Quantity"] > 0) & (merged_data[customer] == 0)].tolist())[:5]
    
    recommendations_consine[customer] = products_cos

In [39]:
user_recommendations_cos = pd.DataFrame.from_dict(recommendations_consine, orient='index').reset_index()
user_recommendations_cos.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recommendations_cos.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,Grapes - Red,Clam Nectar,"Crab - Dungeness, Whole",Coffee - Irish Cream,Vol Au Vents
1,59399,Grapes - Red,Clam Nectar,"Crab - Dungeness, Whole",Pastry - Butterscotch Baked,Coffee - Irish Cream
2,92168,Grapes - Red,Clam Nectar,"Crab - Dungeness, Whole",Pastry - Butterscotch Baked,Coffee - Irish Cream
3,49159,Grapes - Red,Clam Nectar,Pastry - Butterscotch Baked,Coffee - Irish Cream,Vol Au Vents
4,18441,Grapes - Red,Clam Nectar,Pastry - Butterscotch Baked,Coffee - Irish Cream,Vol Au Vents
